<h1>코로나 이전과 코로나 이후의 지하철 사용량을 비교

In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 37 kB/s 
     |████████████████████████████████| 198 kB 51.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=92f0b005019dad70f666c1035c0fe3dec37016b38d9517bccd8bf117b380b7ef
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
import pandas as pd
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
df = spark.read.parquet('/content/drive/MyDrive/data/all_metro')
df.show()

+----------+----------+------------+------------+------------+--------+
|  사용일자|    노선명|        역명|승차총승객수|하차총승객수|이용객수|
+----------+----------+------------+------------+------------+--------+
|2018-01-10|수인분당선|압구정로데오|       16346|       18420|   34766|
|2018-01-10|수인분당선|      서울숲|        7914|        7922|   15836|
|2018-01-10|수인분당선|        수원|        9461|        6280|   15741|
|2018-01-10|수인분당선|        수서|       15403|       15627|   31030|
|2018-01-10|수인분당선|  대모산입구|        3431|        3182|    6613|
|2018-01-10|수인분당선|      개포동|        3582|        3723|    7305|
|2018-01-10|수인분당선|        구룡|        2105|        2005|    4110|
|2018-01-10|수인분당선|        도곡|        7855|        8147|   16002|
|2018-01-10|수인분당선|        한티|       16276|       17214|   33490|
|2018-01-10|수인분당선|        선릉|       10792|       22141|   32933|
|2018-01-10|     4호선|        범계|       32318|       30961|   63279|
|2018-01-10|     4호선|        평촌|       19691|       19258|   38949|
|2018-01-10|     4호선|      인덕원| 

In [ ]:
import pyspark.sql.types as T
import pyspark.sql.functions as F

데이터 정렬

In [ ]:
df = df.sort(['사용일자','노선명'],ascending=True)
df.show()

+----------+------+--------------+------------+------------+--------+
|  사용일자|노선명|          역명|승차총승객수|하차총승객수|이용객수|
+----------+------+--------------+------------+------------+--------+
|2018-01-01| 1호선|        소요산|        4002|        3341|    7343|
|2018-01-01| 1호선|        신이문|        4924|        4538|    9462|
|2018-01-01| 1호선|        동두천|        1248|        1800|    3048|
|2018-01-01| 1호선|        서울역|       40213|       34505|   74718|
|2018-01-01| 1호선|          보산|        1308|        1658|    2966|
|2018-01-01| 1호선|          배방|         654|         909|    1563|
|2018-01-01| 1호선|    동두천중앙|        2430|        2298|    4728|
|2018-01-01| 1호선|쌍용(나사렛대)|        1324|        1450|    2774|
|2018-01-01| 1호선|          지행|        4390|        4399|    8789|
|2018-01-01| 1호선|        의정부|       14452|       15277|   29729|
|2018-01-01| 1호선|          회룡|        6066|        6012|   12078|
|2018-01-01| 1호선|신창(순천향대)|        1101|        1277|    2378|
|2018-01-01| 1호선|        망월사|        38

코로나 이후의 기간이 더 짧기 때문에 이전과 이후의 기간을 같게 맞춰줄 필요가 있다.<br>
먼저 첫 확진자 발생일(2020-01-20)을 기준으로 코로나 이후 며칠의 데이터가 있는지 확인한다.

In [ ]:
df.where(F.col('사용일자')>'2020-01-19').select('사용일자').distinct().count()

620

2020-01-20 부터 620일이 경과했으므로, 우리는 이 이전의 620일의 데이터를 다룰 것이다. 코로나 이전의 데이터를 어느 시점부터 쓸 것인지 결정하기위해 계산을 해주었다

In [ ]:
from datetime import datetime, timedelta

datetime.strptime("2020-01-19", '%Y-%m-%d') - timedelta(days=620)

datetime.datetime(2018, 5, 9, 0, 0)

사람은 실수하는 생물이므로 한번 더 검사하여 이전과 이후의 기간이 같은지 확인했다.

In [ ]:
df.where(F.col('사용일자')>'2018-05-09').select('사용일자').distinct().count()

1240

2018-05-09 이후의 데이터를 이용하면 정확히 620일씩 나눌 수 있음을 확인했으므로 필요없는 데이터들을 제거해준다. 

In [ ]:
df = df.filter(df.사용일자 >'2018-05-09')

In [ ]:
df.show()

+----------+------+--------------+------------+------------+--------+
|  사용일자|노선명|          역명|승차총승객수|하차총승객수|이용객수|
+----------+------+--------------+------------+------------+--------+
|2018-05-10| 1호선|          방학|       11851|       11212|   23063|
|2018-05-10| 1호선|        도봉산|        9797|        5975|   15772|
|2018-05-10| 1호선|          양주|       10985|        9892|   20877|
|2018-05-10| 1호선|        망월사|       12625|       12551|   25176|
|2018-05-10| 1호선|        서울역|       58660|       56392|  115052|
|2018-05-10| 1호선|쌍용(나사렛대)|        3726|        3241|    6967|
|2018-05-10| 1호선|          도봉|        7678|        7398|   15076|
|2018-05-10| 1호선|          보산|        1883|        1761|    3644|
|2018-05-10| 1호선|          지행|        8774|        8641|   17415|
|2018-05-10| 1호선|          아산|        3772|        3067|    6839|
|2018-05-10| 1호선|    동두천중앙|        4527|        4322|    8849|
|2018-05-10| 1호선|          봉명|        1712|        1666|    3378|
|2018-05-10| 1호선|        광운대|    

이제 코로나 이전과 이후를 구분하기 위해 새로운 컬럼을 만들어준다.<br>
새 칼럼의 데이터는 이용자 차이를 구한 후에는 필요가 없으므로 '이전'과 '이후'같은 문자열 데이터 대신, 차이를 구할 때 활용하기 위해 1과 -1로 설정하였다.

In [ ]:
df = df.withColumn('코로나', F.when(df.사용일자 > '2020-01-19', -1).otherwise(1))

In [ ]:
df.show()

+----------+------+--------------+------------+------------+--------+------+
|  사용일자|노선명|          역명|승차총승객수|하차총승객수|이용객수|코로나|
+----------+------+--------------+------------+------------+--------+------+
|2018-05-10| 1호선|          방학|       11851|       11212|   23063|     1|
|2018-05-10| 1호선|        도봉산|        9797|        5975|   15772|     1|
|2018-05-10| 1호선|          양주|       10985|        9892|   20877|     1|
|2018-05-10| 1호선|        망월사|       12625|       12551|   25176|     1|
|2018-05-10| 1호선|        서울역|       58660|       56392|  115052|     1|
|2018-05-10| 1호선|쌍용(나사렛대)|        3726|        3241|    6967|     1|
|2018-05-10| 1호선|          도봉|        7678|        7398|   15076|     1|
|2018-05-10| 1호선|          보산|        1883|        1761|    3644|     1|
|2018-05-10| 1호선|          지행|        8774|        8641|   17415|     1|
|2018-05-10| 1호선|          아산|        3772|        3067|    6839|     1|
|2018-05-10| 1호선|    동두천중앙|        4527|        4322|    8849|     1|
|201

이제 '코로나'컬럼을 통해 코로나 이전과 이후 시기를 구분할 수 있으므로 그룹을 나누어 준다.

In [ ]:
df2 = df.groupby('`코로나`', '`역명`').agg(F.sum('`승차총승객수`').alias('역별이용자수')).orderBy(F.desc('`역명`'))

In [ ]:
df2.show()

+------+--------------------+------------+
|코로나|                역명|역별이용자수|
+------+--------------------+------------+
|     1|    흑석(중앙대입구)|     6155347|
|    -1|    흑석(중앙대입구)|     3816069|
|     1|          효창공원앞|     6218741|
|    -1|          효창공원앞|     4826514|
|    -1|    회현(남대문시장)|    11283181|
|     1|    회현(남대문시장)|    18777734|
|     1|                회룡|     8164715|
|    -1|                회룡|     6276987|
|     1|                회기|    18265333|
|    -1|                회기|    12044998|
|    -1|                화정|     8525525|
|     1|                화정|    12214324|
|     1|                화전|     1603029|
|    -1|                화전|      822115|
|    -1|                화서|     3647735|
|     1|                화서|     5251906|
|     1|화랑대(서울여대입구)|     8048587|
|    -1|화랑대(서울여대입구)|     5573961|
|     1|                화곡|    18541001|
|    -1|                화곡|    14273259|
+------+--------------------+------------+
only showing top 20 rows



이제 코로나 칼럼과 역별 이용자 수를 곱하여 역별로 sum을 해주면 코로나 이전과 이후의 이용자수 추이를 알 수 있다.

In [ ]:
df2 = df2.withColumn('역별이용자수차이', F.col("역별이용자수")*F.col("코로나"))
df2.show()

+------+--------------------+------------+----------------+
|코로나|                역명|역별이용자수|역별이용자수차이|
+------+--------------------+------------+----------------+
|     1|    흑석(중앙대입구)|     6155347|         6155347|
|    -1|    흑석(중앙대입구)|     3816069|        -3816069|
|     1|          효창공원앞|     6218741|         6218741|
|    -1|          효창공원앞|     4826514|        -4826514|
|    -1|    회현(남대문시장)|    11283181|       -11283181|
|     1|    회현(남대문시장)|    18777734|        18777734|
|     1|                회룡|     8164715|         8164715|
|    -1|                회룡|     6276987|        -6276987|
|     1|                회기|    18265333|        18265333|
|    -1|                회기|    12044998|       -12044998|
|    -1|                화정|     8525525|        -8525525|
|     1|                화정|    12214324|        12214324|
|     1|                화전|     1603029|         1603029|
|    -1|                화전|      822115|         -822115|
|    -1|                화서|     3647735|        -3647

In [ ]:
df2 = df2.groupby('`역명`').agg(F.sum('`역별이용자수차이`').alias('승차량차이')).orderBy(F.asc('`역명`'))

계산이 끝난 후의 dataframe을 확인하니 강일역의 이용자수가 코로나 이후에 오히려 증가했다고 나왔다.<br>
이런 케이스가 더 있는지 where문을 활용해 살펴보았다.

In [ ]:
df2.show()

+------------------+----------+
|              역명|승차량차이|
+------------------+----------+
|      4.19민주묘지|    516161|
|              가능|   1173528|
|          가락시장|   2651961|
|    가산디지털단지|   4640157|
|              가양|   2815309|
|            가오리|    508160|
|              가좌|    691542|
|            가천대|   2259788|
|              가평|    595605|
|              간석|   1163178|
|              갈매|    164786|
|              강남|  20206633|
|          강남구청|   2379388|
|              강동|   2811414|
|          강동구청|   1478547|
|              강매|    280532|
|강변(동서울터미널)|  11930742|
|              강일|   -503789|
|              강촌|    143819|
|              개롱|   1104870|
+------------------+----------+
only showing top 20 rows



아래와 같이 이용자수가 역으로 늘어난 역은 총 26개이다.<br>
이러한 결과가 나온 이유는 이 역들이 강일역처럼 개통된지 얼마되지않았거나, 낙성대(강감찬)역처럼 중간에 역명이 바뀌었기 때문이다.<br>
이제 숨어있는 잘못된 데이터를 찾아냈으니 수정해주어야 한다.

In [ ]:
df2.where(F.col('승차량차이')<0).select('역명').show(26)

+-----------------------+
|                   역명|
+-----------------------+
|                   강일|
|                   고색|
|         낙성대(강감찬)|
|동대문역사문화공원(DDP)|
|               둔촌오륜|
|                   마곡|
|   마곡나루(서울식물원)|
|                   미사|
|                   사리|
|                   삼전|
|                 상일동|
|               석촌고분|
|               송파나루|
|                   신내|
|                   야목|
|                   어천|
|                 오목천|
|   용마산(용마폭포공원)|
|                 임진강|
|           중앙보훈병원|
|                   지축|
|               평택지제|
|             하남검단산|
|    하남시청(덕풍·신장)|
|               하남풍산|
|               한성백제|
+-----------------------+



처음엔 모든 정거장에 대한 정보를 검색하여 문제를 해결하려 하였다.

In [ ]:
df = spark.read.parquet('/content/drive/MyDrive/data/all_metro')

In [ ]:
df = df.filter(df.사용일자 >'2018-05-09')

In [ ]:
df.select('역명').distinct().count()

525

그러나 확인하여야 하는 역명이 525개나 되어 이걸 일일히 확인하는것은 시간이 오래걸리고 위함부담도 크다.<br>
따라서 코로나 이전과 이후 그룹으로 나누었을 때, 역에 대한 row의 개수가 차이나는 역들을 먼저 조사하기로 하였다.

In [ ]:
df = df.withColumn('코로나', F.when(df.사용일자 > '2020-01-19', 1).otherwise(-1))
df.show()

+----------+----------+------------------+------------+------------+--------+------+
|  사용일자|    노선명|              역명|승차총승객수|하차총승객수|이용객수|코로나|
+----------+----------+------------------+------------+------------+--------+------+
|2018-05-10|     1호선|            서울역|       58660|       56392|  115052|    -1|
|2018-05-10|     1호선|              오산|       11629|       10918|   22547|    -1|
|2018-05-10|우이신설선|            신설동|        1954|        2114|    4068|    -1|
|2018-05-10|우이신설선|              보문|        1731|        1794|    3525|    -1|
|2018-05-10|우이신설선|성신여대입구(돈암)|        4456|        4396|    8852|    -1|
|2018-05-10|우이신설선|              정릉|        4991|        4472|    9463|    -1|
|2018-05-10|우이신설선|      북한산보국문|        6666|        6318|   12984|    -1|
|2018-05-10|우이신설선|              솔샘|        6207|        5802|   12009|    -1|
|2018-05-10|우이신설선|        삼양사거리|        2586|        2862|    5448|    -1|
|2018-05-10|우이신설선|              삼양|        2314|        2535|    4849|    -1|
|2

In [ ]:
check_df = df.groupby('`역명`', '`코로나`').agg(F.count('`사용일자`').alias('운행기간')).orderBy(F.asc('`역명`'))

In [ ]:
check_df.show()

+--------------+------+--------+
|          역명|코로나|운행기간|
+--------------+------+--------+
|  4.19민주묘지|     1|     620|
|  4.19민주묘지|    -1|     620|
|          가능|    -1|     620|
|          가능|     1|     620|
|      가락시장|     1|    1240|
|      가락시장|    -1|    1240|
|가산디지털단지|    -1|    1240|
|가산디지털단지|     1|    1240|
|          가양|    -1|     620|
|          가양|     1|     620|
|        가오리|    -1|     620|
|        가오리|     1|     620|
|          가좌|    -1|     620|
|          가좌|     1|     620|
|        가천대|     1|     620|
|        가천대|    -1|     620|
|          가평|    -1|     620|
|          가평|     1|     620|
|          간석|    -1|     620|
|          간석|     1|     620|
+--------------+------+--------+
only showing top 20 rows



이제 처음의 방법과 유사한 방법의로 코로나 이전과 이후의 row개수가 차이나는 역을 찾아준다.

In [ ]:
check_df = check_df.withColumn('row개수', F.col("운행기간")*F.col("코로나"))
check_df.show()

+--------------+------+--------+-------+
|          역명|코로나|운행기간|row개수|
+--------------+------+--------+-------+
|  4.19민주묘지|     1|     620|    620|
|  4.19민주묘지|    -1|     620|   -620|
|          가능|    -1|     620|   -620|
|          가능|     1|     620|    620|
|      가락시장|     1|    1240|   1240|
|      가락시장|    -1|    1240|  -1240|
|가산디지털단지|    -1|    1240|  -1240|
|가산디지털단지|     1|    1240|   1240|
|          가양|    -1|     620|   -620|
|          가양|     1|     620|    620|
|        가오리|    -1|     620|   -620|
|        가오리|     1|     620|    620|
|          가좌|    -1|     620|   -620|
|          가좌|     1|     620|    620|
|        가천대|     1|     620|    620|
|        가천대|    -1|     620|   -620|
|          가평|    -1|     620|   -620|
|          가평|     1|     620|    620|
|          간석|    -1|     620|   -620|
|          간석|     1|     620|    620|
+--------------+------+--------+-------+
only showing top 20 rows



In [ ]:
check_df = check_df.groupby('`역명`').agg(F.sum('`row개수`').alias('row차이')).orderBy(F.asc('`역명`'))

이제 row차이가 0이 아닌 역들을 골라준다.<br>
확인 결과 총 44개의 역이 검색되었다.<br>
row차이가 양수라는 뜻은 코로나 이후의 row가 더 많다는 뜻이다.

In [ ]:
check_df.where(F.col('row차이')!=0).select('역명', 'row차이').show(44)

+-----------------------+-------+
|                   역명|row차이|
+-----------------------+-------+
|                   강일|    188|
|                   검암|    -53|
|                   계양|    -45|
|                   고색|    384|
|                   공덕|     -5|
|               김포공항|   -112|
|                 낙성대|   -569|
|         낙성대(강감찬)|    569|
|     동대문역사문화공원|  -1707|
|동대문역사문화공원(DDP)|   1707|
|               둔촌오륜|    205|
|       디지털미디어시티|     -9|
|               마곡나루|   -996|
|   마곡나루(서울식물원)|   1138|
|                   미사|    419|
|                   복정|    -77|
|                   부발|     -1|
|                   사리|    384|
|                   삼전|    205|
|                 서울역|   -141|
|                   석촌|    205|
|               석촌고분|    205|
|             세종대왕릉|     -1|
|               송파나루|    205|
|                   신내|     88|
|                   야목|    384|
|                   어천|    384|
|                 연신내|    -69|
|                 오목천|    384|
|   올림픽공원(한국체대)|    20

아래는 각 역에 대한 정보를 확인하기 위한 코드이다.

In [ ]:
df.where(F.col('역명')=='홍대입구').select('사용일자', '노선명', '승차총승객수').orderBy(F.asc('`사용일자`')).show()

+----------+----------+------------+
|  사용일자|    노선명|승차총승객수|
+----------+----------+------------+
|2018-05-10|  공항철도|        9730|
|2018-05-10|     2호선|       79750|
|2018-05-10|경의중앙선|        1827|
|2018-05-11|경의중앙선|        2501|
|2018-05-11|     2호선|       90696|
|2018-05-11|  공항철도|       11431|
|2018-05-12|경의중앙선|        1878|
|2018-05-12|  공항철도|       10392|
|2018-05-12|     2호선|       86725|
|2018-05-13|  공항철도|       10523|
|2018-05-13|     2호선|       82739|
|2018-05-13|경의중앙선|        1954|
|2018-05-14|     2호선|       72401|
|2018-05-14|경의중앙선|        1724|
|2018-05-14|  공항철도|        9670|
|2018-05-15|     2호선|       76638|
|2018-05-15|  공항철도|        9704|
|2018-05-15|경의중앙선|        1949|
|2018-05-16|  공항철도|        8919|
|2018-05-16|     2호선|       68306|
+----------+----------+------------+
only showing top 20 rows



44개 역에 대한 조사결과는 다음과 같다<br>

|역명|이유|
|--|--|
|                   강일|    2021년 3월 27일부터 운행하였다.|
|                   검암|실제로 검암역을 지나지 않는 경의중앙선이 집계가 되어있었다.|
|                   계양|실제로 계양역을 지나지 않는 경의중앙선이 집계가 되어있었다.|
|                   고색|2020년 9월 12일부터 운행하였다.|
|                   공덕|경의선과 중앙선을 경의중앙선으로 통합하면서 차이가 발생했다.|
|               김포공항|실제로 김포공항역을 지나지 않는 경의중앙선이 집계가 되어있었다.|
|                 낙성대|역명이 변경되었다.|
|         낙성대(강감찬)|역명이 변경되었다.|
|     동대문역사문화공원|역명이 변경되었다.|
|동대문역사문화공원(DDP)|역명이 변경되었다.|
|               둔촌오륜|2018년 12월 1일부터 운행하였다.|
|       디지털미디어시티|경의선과 중앙선을 경의중앙선으로 통합하면서 차이가 발생했다.|
|               마곡나루|역명이 변경되었다. + 2018년 9월 29일부터 공항철도 노선이 추가로 개통되었다.|
|   마곡나루(서울식물원)|역명이 변경되었다. + 2018년 9월 29일부터 공항철도 노선이 추가로 개통되었다.|
|                   미사|2020년 8월 8일부터 운행하였다.|
|                   복정|수인선과 분당선을 수인분당선으로 통합하면서 차이가 발생하였다.|
|                   부발|2020년 8월 3월에 큰 폭우가 내려 운행이 잠시 중단되었었다.|
|                   사리|2020년 9월 12일부터 운행을 시작하였다.|
|                   삼전|2018년 12월 1일부터 운행을 시작하였다.|
|                 서울역|경의선과 중앙선을 경의중앙선으로 통합하면서 차이가 발생했다.|
|                   석촌|2018년 12월 1일부터 운행을 시작하였다.|
|               석촌고분|2018년 12월 1일부터 운행을 시작하였다.|
|             세종대왕릉|2020년 8월 3월에 큰 폭우가 내려 운행이 잠시 중단되었었다.|
|               송파나루|2018년 12월 1일부터 운행을 시작하였다.|
|                   신내|2019년 12월 21일부터 6호선 노선이 추가로 개통되었다.|
|                   야목|2020년 9월 12일부터 운행을 시작하였다.|
|                   어천|2020년 9월 12일부터 운행을 시작하였다.|
|                 연신내|이 역은 개찰구가 따로 마련되어 있지 않기때문에 모든 승하차인원이 3호선으로 집계된다. 때문에 6호선으로 집계되는 항목을 따로 처리해줘야한다.|
|                 오목천|2020년 9월 12일부터 운행을 시작하였다.|
|   올림픽공원(한국체대)|2018년 12월 1일부터 9호선 노선이 추가로 개통되었다.|
|                 용마산|역명이 변경되었다.|
|   용마산(용마폭포공원)|역명이 변경되었다.|
|                 임진강|2020년 3월 28일부터 운행을 시작하였다.|
|           중앙보훈병원|2018년 12월 1일부터 운행을 시작하였다.|
|                   지제|역명이 변경되었다.|
|                   지축|일산선 노선을 3호선으로 편입하면서 차이가 발생하였다.|
|                   창동|이 역은 개찰구가 따로 마련되어 있지 않기때문에 모든 승하차인원이 4호선으로 집계된다. 때문에 1호선으로 집계되는 항목을 따로 처리해줘야한다.|
|                 충무로|이 역은 개찰구가 따로 마련되어 있지 않기때문에 모든 승하차인원이 4호선으로 집계된다. 때문에 3호선으로 집계되는 항목을 따로 처리해줘야한다.|
|               평택지제|역명이 변경되었다.|
|             하남검단산|2021년 3월 27일부터 운행을 시작하였다.|
|    하남시청(덕풍·신장)|2021년 3월 27일부터 운행을 시작하였다.|
|               하남풍산|2020년 8월 8일부터 운행을 시작하였다.|
|               한성백제|2018년 12월 1일부터 운행을 시작하였다.|
|               홍대입구|경의선과 중앙선을 경의중앙선으로 통합하면서 차이가 발생했다.|

해당역들의 이상원인은 다음과 같다.<br>
case1. 같은 사용일자에 동일한 노선명이 들어감 : 기타 노선 통합과정에서 나오는 row차이<br>
case2-1. 코로나 이후의 row가 비정상적으로 많음 : 개통한지 얼마 되지않은 노선<br>
case2-2. 최근에 추가 개통된 환승노선<br>
case3. 비슷한 이름의 역명이 2개 존재 : 기간 내에 역명이 변경된 사례<br>
case4. 위의 case들에 속하진 않지만 차이가 나는 경우 : 해당 노선이 지나지 않는 역인데 기록이된 사례<br>
case5. 특이케이스 : 2020년 8월 3일에 내린 큰 폭우로 하루동안 운행을 멈춘 사례<br>
case6. 특이케이스 : 개찰구가 한 종류밖에 없는 환승역. 모든 이용자들이 해당 개찰구의 노선으로 기록되어야 하나 다른 노선으로 기록된 표본이 있는 사례

case1과 case2-2는 정상적인 현상이므로 무시한다.<br>
case2-1은 우리가 비교하려는 정보(코로나 이전과 이후의 이용량 차이)를 얻을 수 없으므로 제거해준다.(12359 rows)

In [ ]:
late_open = ['강일', '고색', '둔촌오륜', '미사', '사리', '삼전', '석촌', '석촌고분', '송파나루', '야목', '어천', '오목천', '임진강', '중앙보훈병원', '하남검단산', '하남시청(덕풍·신장)', '하남풍산', '한성백제']

In [ ]:
df = df.filter(~df.역명.isin(late_open))

In [ ]:
df.show()

+----------+----------+------------------+------------+------------+--------+------+
|  사용일자|    노선명|              역명|승차총승객수|하차총승객수|이용객수|코로나|
+----------+----------+------------------+------------+------------+--------+------+
|2018-05-10|     1호선|            서울역|       58660|       56392|  115052|    -1|
|2018-05-10|     1호선|              오산|       11629|       10918|   22547|    -1|
|2018-05-10|우이신설선|            신설동|        1954|        2114|    4068|    -1|
|2018-05-10|우이신설선|              보문|        1731|        1794|    3525|    -1|
|2018-05-10|우이신설선|성신여대입구(돈암)|        4456|        4396|    8852|    -1|
|2018-05-10|우이신설선|              정릉|        4991|        4472|    9463|    -1|
|2018-05-10|우이신설선|      북한산보국문|        6666|        6318|   12984|    -1|
|2018-05-10|우이신설선|              솔샘|        6207|        5802|   12009|    -1|
|2018-05-10|우이신설선|        삼양사거리|        2586|        2862|    5448|    -1|
|2018-05-10|우이신설선|              삼양|        2314|        2535|    4849|    -1|
|2

case3은 이름이 변경된 두 역을 통합해주어야한다.

In [ ]:
change_target = ['낙성대', '동대문역사문화공원', '마곡나루', '용마산', '지제']

In [ ]:
df = df.withColumn("역명", F.when(df.역명 == "낙성대","낙성대(강감찬)").otherwise(df.역명))
df = df.withColumn("역명", F.when(df.역명 == "동대문역사문화공원","동대문역사문화공원(DDP)").otherwise(df.역명))
df = df.withColumn("역명", F.when(df.역명 == "마곡나루","마곡나루(서울식물원)").otherwise(df.역명))
df = df.withColumn("역명", F.when(df.역명 == "용마산","용마산(용마폭포공원)").otherwise(df.역명))
df = df.withColumn("역명", F.when(df.역명 == "지제","평택지제").otherwise(df.역명))

혹시 지워지지 않은 역이 있는지 확인

In [ ]:
df.filter(df.역명.isin(change_target)).show()

+--------+------+----+------------+------------+--------+------+
|사용일자|노선명|역명|승차총승객수|하차총승객수|이용객수|코로나|
+--------+------+----+------------+------------+--------+------+
+--------+------+----+------------+------------+--------+------+



case4는 통상적으로 있어선 안되는 데이터 이므로 제거를 해주어야 한다.(444 rows)
case5는 

In [ ]:
strange_target = ['검암', '계양', '김포공항']

In [ ]:
df = df.filter(~(df.노선명.isin('경의중앙선') & df.역명.isin(strange_target)))

혹시 지워지지 않은 역이 있는지 확인(모두 경의중앙선이므로 한번에 확인할 수 있다.)

In [ ]:
df.filter(df.역명.isin(strange_target)).filter(df.노선명.isin('경의중앙선')).show()

+--------+------+----+------------+------------+--------+------+
|사용일자|노선명|역명|승차총승객수|하차총승객수|이용객수|코로나|
+--------+------+----+------------+------------+--------+------+
+--------+------+----+------------+------------+--------+------+



이제 row개수를 확인하여 제대로 처리가 되었는지 검토한다.<br>
우리는 총 734412 row 중 해당 역이 늦게 개통하여 사용할 수 없는 12359 row와 해당노선이 지나지 않는 역이 기록된 444 row를 제거하였다.<br>
즉 총 721609의 row가 남아있어야한다.

In [ ]:
df.count()

721609

특이 케이스(case5, case6)에서 잘못집계되는 지하철역(운행중지, 집계오류)의 존재를 인지하게 되었다.<br>
filter를 통해 단순히 이용인원이 일정 수 이하인 노선을 제거할수도 있지만, 단순히 이용객이 적을 뿐인 노선일수도 있으므로 최소한의 검증을 해보았다.<br>
통상적으로 최소한 10명 이상의 인원이 지하철역을 이용했으리라 기대되므로 10명을 기준으로 설정하였다.

In [ ]:
df.filter(df.이용객수<10).select('노선명', '역명').distinct().show()

+----------+----------------+
|    노선명|            역명|
+----------+----------------+
|    경강선|            여주|
|경의중앙선|디지털미디어시티|
|    경강선|      세종대왕릉|
|경의중앙선|            공덕|
|수인분당선|            복정|
|경의중앙선|          서울역|
|경의중앙선|        홍대입구|
|     6호선|          연신내|
|     3호선|          충무로|
|     1호선|            창동|
|     6호선|            신내|
|    경강선|            이천|
|     3호선|            지축|
|    경강선|            부발|
+----------+----------------+



총 14개의 노선-역의 조합이 일간 사용자가 10명 미만으로 집계되었다.<br>
이제 원본파일과 데이터프레임을 이용하여 이상치의 원인을 알아보면 된다.

In [ ]:
# 각 역의 정보를 확인해보기 위한 코드
df.filter(df.노선명.isin('1호선')).filter(df.역명.isin('창동')).show()

+----------+------+----+------------+------------+--------+------+
|  사용일자|노선명|역명|승차총승객수|하차총승객수|이용객수|코로나|
+----------+------+----+------------+------------+--------+------+
|2018-05-10| 1호선|창동|           1|           0|       1|    -1|
|2018-05-12| 1호선|창동|           2|           0|       2|    -1|
|2018-05-13| 1호선|창동|           2|           0|       2|    -1|
|2018-05-16| 1호선|창동|           1|           0|       1|    -1|
|2018-05-21| 1호선|창동|           1|           0|       1|    -1|
|2018-05-25| 1호선|창동|           3|           0|       3|    -1|
|2018-05-27| 1호선|창동|           1|           0|       1|    -1|
|2018-05-30| 1호선|창동|           3|           0|       3|    -1|
|2018-05-31| 1호선|창동|           1|           0|       1|    -1|
|2018-06-01| 1호선|창동|           1|           0|       1|    -1|
|2018-06-02| 1호선|창동|           3|           0|       3|    -1|
|2018-06-04| 1호선|창동|           1|           0|       1|    -1|
|2018-06-09| 1호선|창동|           1|           0|       1|    -1|
|2018-06

확인결과 원인은 다음과 같았다.

|노선명|역명|원인|
|--|--|--|
|3호선|지축|집계오류(같은날에 2건이 집계됨)|
|경의중앙선|서울역|집계오류(같은날에 2건이 집계됨)|
|경의중앙선|공덕|집계오류(같은날에 2건이 집계됨)|
|경의중앙선|디지털미디어시티|집계오류(같은날에 2건이 집계됨)|
|경의중앙선|홍대입구|집계오류(같은날에 2건이 집계됨)|
|1호선|창동|해당 노선의 개찰구가 없는데 집계됨|
|3호선|충무로|해당 노선의 개찰구가 없는데 집계됨|
|6호선|신내|해당 노선의 개찰구가 없는데 집계됨|
|6호선|연신내|해당 노선의 개찰구가 없는데 집계됨|
|수인분당선|복정|해당 노선의 개찰구가 없는데 집계됨|
|경강선|부발|폭우로 인한 운행 중지|
|경강선|세종대왕릉|폭우로 인한 운행 중지|
|경강선|여주|폭우로 인한 운행 중지|
|경강선|이천|폭우로 인한 운행 중지|


3호선 지축역~ 경의중앙선 홍대입구역은 원래 매일 하나의 이용객 데이터가 존재하여야하는데 2개 이상의 데이터가 존재한 경우다.<br>
확인결과 10명 미만의 이용객이 추가로 집계된 데이터가 298건 존재하였다.<br>
이는 비정상적인 데이터이므로 10명미만의 데이터를 삭제를 해주었다.

In [ ]:
strange_station = ['지축', '서울역', '공덕', '디지털미디어시티', '홍대입구']

In [ ]:
df = df.filter(~(df.노선명.isin(['3호선', '경의중앙선']) & df.역명.isin(strange_station) & df.이용객수.isin(list(range(10)))))

In [ ]:
df.count()

721311

1호선 창동역 ~ 6호선 연신내역은 환승역이지만 한 종류의 개찰구 밖에 없어 단일노선만 집계되어야 하는 역이다.<br>
그러나 확인결과 다른노선의 사용인원도 존재하였다.<br>
이는 비정상적인 데이터이므로 전체를 삭제하였다.

In [ ]:
df.filter((df.노선명.isin('1호선') & df.역명.isin('창동'))).count()

427

In [ ]:
df = df.filter(~(df.노선명.isin('1호선') & df.역명.isin('창동')))

In [ ]:
df.filter((df.노선명.isin('3호선') & df.역명.isin('충무로'))).count()

857

In [ ]:
df = df.filter(~(df.노선명.isin('3호선') & df.역명.isin('충무로')))

In [ ]:
df.filter((df.노선명.isin('6호선') & df.역명.isin(['신내', '연신내']))).count()

665

In [ ]:
df = df.filter(~(df.노선명.isin('6호선') & df.역명.isin(['신내', '연신내'])))

In [ ]:
df.filter((df.노선명.isin('수인분당선') & df.역명.isin('복정'))).count()

477

In [ ]:
df = df.filter(~(df.노선명.isin('수인분당선') & df.역명.isin('복정')))

경강선 노선은 2020년 8월 3일의 호우로 인해 운행이 잠시 중단되었었다.<br>
회귀를 통해 결측치를 메워볼까해서 팀원들의 의견을 모아본 결과, 결과분석에 주는 영향이 미미하므로 제거하기로 하였다.

In [ ]:
df.filter((df.이용객수.isin(list(range(10))) & df.노선명.isin('경강선') & df.역명.isin(['이천', '부발', '세종대왕릉', '여주']))).count()

6

In [ ]:
df = df.filter(~(df.이용객수.isin(list(range(10))) & df.노선명.isin('경강선') & df.역명.isin(['이천', '부발', '세종대왕릉', '여주'])))

남아있는 이상치(이용객수 10명 미만) 확인

In [ ]:
df.filter(df.이용객수<10).show()

+--------+------+----+------------+------------+--------+------+
|사용일자|노선명|역명|승차총승객수|하차총승객수|이용객수|코로나|
+--------+------+----+------------+------------+--------+------+
+--------+------+----+------------+------------+--------+------+



In [ ]:
# 중간저장을 위한 코드
# df.write.parquet('/content/drive/MyDrive/data/중간저장/preprocessing')

In [ ]:
# 중간저장을 불러오기 위한 코드
df = spark.read.format("parquet").option("header", True).load("/content/drive/MyDrive/data/중간저장/preprocessing")
df

DataFrame[사용일자: date, 노선명: string, 역명: string, 승차총승객수: int, 하차총승객수: int, 이용객수: int, 코로나: int]

In [ ]:
df.show()

+----------+----------+------------------+------------+------------+--------+------+
|  사용일자|    노선명|              역명|승차총승객수|하차총승객수|이용객수|코로나|
+----------+----------+------------------+------------+------------+--------+------+
|2018-05-10|     1호선|            서울역|       58660|       56392|  115052|    -1|
|2018-05-10|     1호선|              오산|       11629|       10918|   22547|    -1|
|2018-05-10|우이신설선|            신설동|        1954|        2114|    4068|    -1|
|2018-05-10|우이신설선|              보문|        1731|        1794|    3525|    -1|
|2018-05-10|우이신설선|성신여대입구(돈암)|        4456|        4396|    8852|    -1|
|2018-05-10|우이신설선|              정릉|        4991|        4472|    9463|    -1|
|2018-05-10|우이신설선|      북한산보국문|        6666|        6318|   12984|    -1|
|2018-05-10|우이신설선|              솔샘|        6207|        5802|   12009|    -1|
|2018-05-10|우이신설선|        삼양사거리|        2586|        2862|    5448|    -1|
|2018-05-10|우이신설선|              삼양|        2314|        2535|    4849|    -1|
|2

지하철 위도, 경도를 저장한 파일을 불러옴

In [ ]:
station_locate = spark.read.format("csv").option("header", True).load("/content/drive/MyDrive/data/station_coordinate.csv", encoding='utf-8')

In [ ]:
# 데이터확인. line: 노선명 | name: 역명 | code: 역코드 | lat: 위도 | lng: 경도
station_locate

DataFrame[line: string, name: string, code: string, lat: string, lng: string]

In [ ]:
# 불러온 위치 데이터에서는 역명을 기록한 칼럼의 이름이 'name'이라 되어있다. 후에 join을 하기위해 컬럼명을 '역명'이라고 통일해준다.
station_locate = station_locate.withColumnRenamed("name", "역명")

In [ ]:
# 필요한 칼럼만 추려낸다.
station_locate = station_locate.select('역명', 'lat', 'lng').distinct()

In [ ]:
station_locate.count()

616

In [ ]:
# 위치 데이터가 얼추 준비되었으므로 다시 역별 이용자수 데이터로 돌아옴.
# 코로나 이전과 이후의 이용자 차이를 sum함수를 구하기위해 -1과 1로 구분된 '코로나' 칼럼을 이용함
# 역별이용자수 칼럼은 코로나 이전의 이용자수를 음수로, 코로나 이후의 이용자수를 양수로 저장했다.
df2 = df.withColumn('역별이용자수', F.col("이용객수")*F.col("코로나"))

In [ ]:
# sum을 이용해 코로나 이전과 이후의 역별이용자수차이를 구해주어 '이용자수차이' 칼럼에 저장한다.
df2 = df2.groupby('`역명`').agg(F.sum('`역별이용자수`').alias('이용자수차이')).orderBy(F.asc('`역명`'))

이제 코로나 이후 이용자가 증가한 역을 조사해보자(놓친 오류가 있을수도 있으므로)

In [ ]:
df2.where(F.col('이용자수차이')>0).show()

+--------------------+------------+
|                역명|이용자수차이|
+--------------------+------------+
|                마곡|      524605|
|마곡나루(서울식물원)|     1567607|
|              상일동|      491220|
|                신내|      489762|
|                지축|     1805136|
+--------------------+------------+



조사결과 해당 역들의 이용객 증가 사유는 다음과 같다.

|역명|이유|
|--|--|
|마곡|최근에도 개발중이 진행중인 지구로 이용자수가 점점 늘어나고있음|
|마곡나루(서울식물원)|개발중인 지구 + 공항철도의 추가 개통으로 이용자수가 증가함|
|상일동|출퇴근시간대에 미사강변도시나 강일동에서 상일동역을 이용하는데 그 입주민 수가 증가함(예측)|
|신내|6호선의 추가 개통으로 이용자수가 증가함|
|지축|기존엔 이용객이 저조한 편이였으나 주변일대의 개발로 이용자수가 늘어남|

데이터는 이상이 없다 판단되므로, 이제 '역명' 칼럼에 left outer join을 이용해 위도와 경도 컬럼을 추가해 주자.

In [ ]:
df = df2.join(station_locate, on='역명', how='left')

기존 데이터와 위도경도 데이터의 역명이 다르면(혹은 정보가 없다면) Null값이 생성되었을 것이다. 이러한 역들을 파악해보자.

In [ ]:
df.where(F.col('lat').isNull()==True).count()

63

예상보다 많은 역이 나와 당황했다... 어떤 역들이 있는지 확인하여 처리하자.

In [ ]:
df.where(F.col('lat').isNull()==True).select('역명').show(63)

+-----------------------------+
|                         역명|
+-----------------------------+
|           충정로(경기대입구)|
|             안암(고대병원앞)|
|               천호(풍납토성)|
|               신창(순천향대)|
|           봉화산(서울의료원)|
|         광화문(세종문화회관)|
|         마곡나루(서울식물원)|
|         어린이대공원(세종대)|
|         경복궁(정부서울청사)|
|         상봉(시외버스터미널)|
|             왕십리(성동구청)|
|            교대(법원.검찰청)|
|남한산성입구(성남법원.검찰청)|
|               수유(강북구청)|
|           월드컵경기장(성산)|
|               쌍용(나사렛대)|
|       청량리(서울시립대입구)|
|           숭실대입구(살피재)|
|             총신대입구(이수)|
|                 동작(현충원)|
|                   새절(신사)|
|           강변(동서울터미널)|
|           몽촌토성(평화의문)|
|     아차산(어린이대공원후문)|
|                 고려대(종암)|
|               증산(명지대앞)|
|                 광흥창(서강)|
|         서울대입구(관악구청)|
|                   군자(능동)|
|         오목교(목동운동장앞)|
|                 신정(은행정)|
|               잠실(송파구청)|
|         미아(서울사이버대학)|
|               삼성(무역센터)|
|               대림(구로구청)|
|             회현(남대문시장)|
|                  

해당 63개 row의 역명에 대해 station_locate를 수정해주자. 대부분 병기역명이므로 처리자체는 어렵지 않을것으로 보인다.

먼저 병기명을 가진 역들부터 처리하겠다.

In [ ]:
# 해당하는 역이 많이므로 딕셔너리를 활용하려한다.(수작업)
duel_name = {'충정로': '충정로(경기대입구)', '안암': '안암(고대병원앞)', '천호': '천호(풍납토성)',
             '신창': '신창(순천향대)', '봉화산': '봉화산(서울의료원)', '광화문': '광화문(세종문화회관)',
             '마곡나루': '마곡나루(서울식물원)', '어린이대공원': '어린이대공원(세종대)', '경복궁': '경복궁(정부서울청사)',
             '상봉': '상봉(시외버스터미널)', '왕십리': '왕십리(성동구청)', '교대': '교대(법원.검찰청)',
             '남한산성입구': '남한산성입구(성남법원.검찰청)', '수유': '수유(강북구청)', '월드컵경기장': '월드컵경기장(성산)',
             '쌍용': '쌍용(나사렛대)', '청량리': '청량리(서울시립대입구)', '숭실대입구': '숭실대입구(살피재)',
             '총신대입구': '총신대입구(이수)', '동작': '동작(현충원)', '새절': '새절(신사)', '강변': '강변(동서울터미널)',
             '몽촌토성': '몽촌토성(평화의문)', '아차산': '아차산(어린이대공원후문)', '고려대': '고려대(종암)', '증산': '증산(명지대앞)',
             '광흥창': '광흥창(서강)', '서울대입구': '서울대입구(관악구청)', '군자': '군자(능동)', '오목교': '오목교(목동운동장앞)',
             '신정': '신정(은행정)', '잠실': '잠실(송파구청)', '미아': '미아(서울사이버대학)', '삼성': '삼성(무역센터)',
             '대림': '대림(구로구청)', '회현': '회현(남대문시장)', '구의': '구의(광진구청)', '용마산': '용마산(용마폭포공원)',
             '남부터미널': '남부터미널(예술의전당)', '화랑대': '화랑대(서울여대입구)', '녹사평': '녹사평(용산구청)',
             '용두': '용두(동대문구청)', '대흥': '대흥(서강대앞)', '낙성대': '낙성대(강감찬)', '양재': '양재(서초구청)',
             '상월곡': '상월곡(한국과학기술연구원)', '숙대입구': '숙대입구(갈월)', '광나루': '광나루(장신대)',
             '온수': '온수(성공회대입구)', '올림픽공원': '올림픽공원(한국체대)', '한성대입구': '한성대입구(삼선교)',
             '이촌': '이촌(국립중앙박물관)', '동대문역사문화공원': '동대문역사문화공원(DDP)', '공릉': '공릉(서울과학기술대)',
             '성신여대입구': '성신여대입구(돈암)', '굽은다리': '굽은다리(강동구민회관앞)', '월곡': '월곡(동덕여대)', '흑석': '흑석(중앙대입구)'}

In [ ]:
# 중간점검을 위한 코드. duel_name_list의 length와 아래 셀의 count결과가 같으면 문제가 없는 것이다.
# 위 딕셔너리를 만드는 중간중간에 실행해주어 실수를 체크한다.
duel_name_list = []
for key in duel_name:
    duel_name_list.append(key)
print(len(duel_name_list))

58


In [ ]:
# station_locate에 duel_name_list 안의 역이 얼마나 존재하는지 체크하는 코드
station_locate.where(F.col('역명').isin(duel_name_list)).count()

58

In [ ]:
# for문을 이용해 withColumn을 반복해주면 세션자체가 먹통이된다.
# withColumn을 여러번 반복해서 사용한것이 원인이라 판단해 딕셔너리와 한번의 withColumn을 이용하여 바꾸어 줬더니 해결되었다.
# create_map을 이용해 station_locate의 역명을 변경해주었다. chain과 create_map에 대한 자세한 내용은 일단 공식문서나 stackoverflow 참고바람. 저도 지금 보는중임
from itertools import chain


mapping_expr = F.create_map([F.lit(x) for x in chain(*duel_name.items())])
station_locate = station_locate.withColumn("역명", F.when(station_locate.역명.isin(duel_name_list), mapping_expr[F.col("역명")]).otherwise(station_locate.역명))

In [ ]:
# 변경이 잘되었나 확인
station_locate.where(F.col('역명').isin(duel_name_list)).select('역명').show()

+----+
|역명|
+----+
+----+



이제 한번 더 left outer join을 하여 null값을 확인해보자.

In [ ]:
df = df2.join(station_locate, on='역명', how='left')

In [ ]:
df.where(F.col('lat').isNull()==True).select('역명').show()

+---------------+
|           역명|
+---------------+
|         서울역|
|   4.19민주묘지|
|인천공항1터미널|
|인천공항2터미널|
|       평택지제|
+---------------+



확인 결과 station_locate에는 '서울역'이 '서울'로, '4.19민주묘지'는 '419민주묘지'로, '평택지제'가 '지제'로 저장되어있었다.<br>
인천공항1터미널과 인청공항2터미널은 원래부터 위치정보가 없었다.<br>
먼저 역명부터 변경해주자

In [ ]:
# 역명을 바꾸기 위한 딕셔너리와 리스트
change_name = {'서울': '서울역', '419민주묘지': '4.19민주묘지', '지제': '평택지제'}
change_name_list = []
for key in change_name:
    change_name_list.append(key)

In [ ]:
mapping_expr = F.create_map([F.lit(x) for x in chain(*change_name.items())])
station_locate = station_locate.withColumn("역명", F.when(station_locate.역명.isin(change_name_list), mapping_expr[F.col("역명")]).otherwise(station_locate.역명))

인천공항1터미널의 좌표는 37.447492, 126.452555이고<br>
인천공항2터미널의 좌표는 37.467407, 126.434142이다.<br>
이 정보를 추가해주자.

In [ ]:
# 1 터미널
station_locate = station_locate.withColumn("lat", F.when(station_locate.역명=='인천공항1터미널', '37.447492').otherwise(station_locate.lat))
station_locate = station_locate.withColumn("lng", F.when(station_locate.역명=='인천공항1터미널', '126.452555').otherwise(station_locate.lng))
# 2터미널
station_locate = station_locate.withColumn("lat", F.when(station_locate.역명=='인천공항2터미널', '37.467407').otherwise(station_locate.lat))
station_locate = station_locate.withColumn("lng", F.when(station_locate.역명=='인천공항2터미널', '126.434142').otherwise(station_locate.lng))

In [ ]:
# 이제 완성된 station_locate을 join을 해준다.
df = df2.join(station_locate, on='역명', how='left')

null값이 아직 있는지 확인

In [ ]:
# 위도
df.where(F.col('lat').isNull()==True).show()

+----+------------+---+---+
|역명|이용자수차이|lat|lng|
+----+------------+---+---+
+----+------------+---+---+



In [ ]:
# 경도
df.where(F.col('lng').isNull()==True).show()

+----+------------+---+---+
|역명|이용자수차이|lat|lng|
+----+------------+---+---+
+----+------------+---+---+



이제 완성된 데이터를 저장하자.

In [ ]:
df.write.parquet('/content/drive/MyDrive/data/contrast_metro_user/user_difference.parquet')

In [ ]:
df = spark.read.format("parquet").option("header", True).load("/content/drive/MyDrive/data/contrast_metro_user/user_difference.parquet")
df.head()

Row(역명='역곡', 이용자수차이=-10650679, lat='37.485178', lng='126.811502')

In [ ]:
df.show()

+------------------+------------+---------+----------+
|              역명|이용자수차이|      lat|       lng|
+------------------+------------+---------+----------+
|              역곡|   -10650679|37.485178|126.811502|
|              혜화|   -21611716|37.582336|127.001844|
|            한양대|    -7469505|37.555273|127.043655|
|              금호|    -2384938|37.548034|127.015872|
|            노량진|   -16360036|37.514219|126.942454|
|              판교|     -633626|37.394761|127.111217|
|            신이문|    -2633818|37.601854|127.067325|
|              신답|     -388636| 37.57004|127.046481|
|              서현|   -10688432|37.385126|127.123592|
|         을지로3가|   -11685301|37.566295| 126.99191|
|              약수|    -5761539| 37.55434|127.010655|
|              평촌|    -4465531|37.394287|126.963883|
|            한대앞|    -2709565|37.309689| 126.85344|
|충정로(경기대입구)|    -5034192|37.559973|126.963672|
|              마두|    -3199123|37.652206| 126.77762|
|        대모산입구|     -592505|37.491373| 127.07272|
|         